In [1]:
import sklearn
import numpy as np

In [2]:
def loadSamples(sample_path):
    '''
    Loads the CSV sample data
    '''
    import csv
    samples = []
    with open(sample_path + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    return samples

In [3]:
def extractSamples(samples, data_path):
    '''
    Extracts needed training data and corresponding measurements
    '''
    image_paths = []
    measurements = []
    for line in samples:
        image_paths.append(data_path + '/' +line[0])    # center_image column
        measurements.append(line[3])                    # steering column
    return image_paths, measurements

In [4]:
import cv2
def generator(samples, batch_size=32):
    '''
    Generate shuffled batch samples on the fly
    '''
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            steerings = []
            for image_path, measurement in batch_samples:
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                images.append(image)
                steerings.append(float(measurement))

            # trim image to only see section with road
            X = np.array(images)
            y = np.array(steerings)
            yield sklearn.utils.shuffle(X, y)

In [5]:
data_path = './recording'
csvData = loadSamples(data_path)
image_paths, measurements = extractSamples(csvData, data_path)
samples = list(zip(image_paths, measurements))

In [12]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = sklearn.model_selection.train_test_split(samples, test_size=0.2)
print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))

Train samples: 108
Validation samples: 28


In [13]:
# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

In [14]:
def nvidiaCNN(input_shape):
    '''
    Define the Nvidia End-to-End CNN architecture
    '''
    from keras.models import Sequential, Model
    from keras.layers import Lambda, Cropping2D, Convolution2D, Flatten, Dense
    model = Sequential()
    print("Input shape : {}".format(input_shape))
    # First normalize the input image with Lambda layer
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    print("Lambda layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # Crop the usefule area
    model.add(Cropping2D(cropping=((70,25), (0,0))))
    print("Cropping layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 1st Convolution layer
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    print("1st Convolution layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 2nd Convolution layer
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))  
    print("2nd Convolution layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 3rd Convolution layer
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    print("3rd Convolution layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 4th Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu'))
    print("4th Convolution layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 5th Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu'))
    print("5th Convolution layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # Flatten layer
    model.add(Flatten())
    print("Flatten layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 1st Fully connected layer
    model.add(Dense(100))
    print("1st Fully connected layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 2nd Fully connected layer
    model.add(Dense(50))
    print("2nd Fully connected layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # 3rd Fully connected layer
    model.add(Dense(10))
    print("3rd Fully connected layer output shape : {}".format(model.layers[-1].output_shape[1:]))

    # Output layer
    model.add(Dense(1))
    print("Output layer output shape : {}".format(model.layers[-1].output_shape[1:]))
    return model


In [15]:
raw_image_shape = (160,320,3)
model = nvidiaCNN(raw_image_shape)

Input shape : (160, 320, 3)
Lambda layer output shape : (160, 320, 3)
Cropping layer output shape : (65, 320, 3)
1st Convolution layer output shape : (31, 158, 24)
2nd Convolution layer output shape : (14, 77, 36)
3rd Convolution layer output shape : (5, 37, 48)
4th Convolution layer output shape : (3, 35, 64)
5th Convolution layer output shape : (1, 33, 64)
Flatten layer output shape : (2112,)
1st Fully connected layer output shape : (100,)
2nd Fully connected layer output shape : (50,)
3rd Fully connected layer output shape : (10,)
Output layer output shape : (1,)


In [16]:
model.compile(loss='mse', optimizer='adam')

In [18]:
history_object = model.fit_generator(train_generator, samples_per_epoch= \
                 len(train_samples), validation_data=validation_generator, \
                 nb_val_samples=len(validation_samples), nb_epoch=1, verbose=1)
model.save('model.h5')

Epoch 1/1
108/108 [==============================] - 21s 194ms/step - loss: 5.4155e-04 - val_loss: 0.0851
